## 빅데이터 실습

### 제주도 핫플레이스 시각화

#### 데이터 수집

##### 셀레니움 사용 자동화

In [26]:
# 필요 라이브러리 사용등록
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By

In [27]:
# 웹 드라이버 스타벅스 웹사이트 오픈
driver = webdriver.Chrome()
url = 'https://www.starbucks.co.kr/store/store_map.do?disp=locale'
driver.get(url)

In [24]:
# 웹드라이버에서 소스 위치 서울링크 클릭(생략)
path = 'div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, path).click()

In [ ]:
# 웹드라이버에서 소스 위치 서울링크 클릭()
name = 'set_sido_cd_btn'
driver.find_element(By.CLASS_NAME, name).click()

In [37]:
# 웹드라이버에서 소스 위치 부산링크 클릭()
name = 'set_sido_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[5].click()

In [40]:
# 웹드라이버에서 소스 위치 전체링크 클릭()
name = 'set_sido_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

In [41]:
name = 'set_gugun_cd_btn'
driver.find_elements(By.CLASS_NAME, name)[0].click()

In [51]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
soup

<html lang="ko"><head>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="website" property="og:type"/>
<meta content="Starbucks" property="og:title"/>
<meta content="https://www.starbucks.co.kr/" property="og:url"/>
<meta content="https://image.istarbucks.co.kr/common/img/kakaotalk.png" property="og:image"/>
<meta content="Starbucks" property="og:description"/>
<title id="titleJoin">Starbucks Korea</title><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-WC8Q3C59QP&amp;cx=c&amp;_slc=1" type="text/javascript"></script><script async="" crossorigin="anonymous" src="https://connect.facebook.net/ko_KR/sdk.js?hash=b0e5e8021cd52c1ec35b6487828538b6"></script><script id="facebook-jssdk" src="//connect.facebook.net/ko_KR/sdk.js"></script><script async="" src="//www.google-analytics.com/analytics.js"></script>

In [43]:
# 검색결과 613개 겟
sbSeoulList = soup.select('li.quickResultLstCon')
len(sbSeoulList)

613

In [44]:
sbSeoulList[0]

<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>

#### 서울매장 리스트 DF

In [48]:
sbSeoulStores = []
for item in sbSeoulList:    #613반복
    name = item.select('strong')[0].text.strip()    # 지점명
    lat = item['data-lat'].strip()  # 위도
    lng = item['data-long'].strip() # 경도
    storeType = item.select('i')[0]['class'][0].split('_')[1]   # 가게종류(일반, 리저브, DT, WT)
    address = str(item.select('p.result_details')[0]).split('<br/>')[0].split('>')[1]   # 주소
    tel = str(item.select('p.result_details')[0]).split('<br/>')[1].split('<')[0]   # 전화번호
    # print(address)
    
    sbSeoulStores.append([name, lat, lng, storeType, address, tel])

In [49]:
# DF로 변경
dfStarBucksSeoul = pd.DataFrame(sbSeoulStores, columns=['매장명', '위도', '경도', '매장타입', '주소', '전화번호'])

In [50]:
dfStarBucksSeoul

,매장명,위도,경도,매장타입,주소,전화번호
0,역삼아레나빌딩,37.501087,127.043069,general,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,general,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,general,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,general,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩,37.494668,127.062583,general,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...,...,...,...
608,사가정역,37.579594,127.087966,general,서울특별시 중랑구 면목로 310,1522-3232
609,상봉역,37.59689,127.08647,general,서울특별시 중랑구 망우로 307 (상봉동),1522-3232
610,묵동,37.615368,127.076633,general,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232
611,양원역,37.6066536267232,127.106359790053,general,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232


In [52]:
# 결측치가 하나도 없음
dfStarBucksSeoul.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 613 entries, 0 to 612
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   매장명     613 non-null    object
 1   위도      613 non-null    object
 2   경도      613 non-null    object
 3   매장타입    613 non-null    object
 4   주소      613 non-null    object
 5   전화번호    613 non-null    object
dtypes: object(6)
memory usage: 28.9+ KB


In [54]:
dfStarBucksSeoul.to_excel('./data/starbucks_seoul_list.xlsx', index=False)

##### 서울 열린데이터 광장 Open API 공공데이터 수집

- 아래방법 동작안함

In [55]:
# 서울 열린 데이터 광장 API 키
SEOUL_API_KEY = '본인 키'

In [56]:
# 서울 열린 데이터 광장 호출 URL
url = f'http://openapi.seoul.go.kr:8088/{SEOUL_API_KEY}/json/SdeTlSccoSigW/1/25'
url

'http://openapi.seoul.go.kr:8088/http://openapi.seoul.go.kr:8088/sample/xml/LOCALDATA_031101_GD/1/5//json/SdeTlSccoSigW/1/25'

##### 행정구역 데이터(위경도 + 구역)
- https://github.com/southkorea/seoul-maps/blob/master/kostat/2013/json/seoul_municipalities_geo_simple.json
